In [1]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation

import numpy as np
import random
import time

In [2]:
#En esta parte estoy creando al agente y definiendo las acciones que puede realizar
class VacumAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)

    # Esta función permite al agente moverse a una posición aleatoria en su "vecindario", me baso en el código que hicimos en "game of life".
    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    # Esta funcion es para cuando el agente llega a una nueva celda que es lo que tiene que hacer, ya sea limpiar la celda o moverse.
    def step(self):
        if self.model.is_dirt(self.pos):
            self.model.clean_dirt(self.pos)
            self.move()
        else:
            self.move()

In [3]:
# En esta parte estamos haciedo el modelo de lo que va a hacer el agente, como se va a mover, cuantas celdas sucias hay, etc.
# Para la generación de la matriz y de como darle los valores de "sucio" a esa matriz, me base en el segretation model.
class VacumModel(Model):
    def __init__(self, width, height, num_agents, max_steps, dirty_cells):
        self.num_agents = num_agents
        self.max_steps = max_steps
        self.dirty_cells = dirty_cells
        self.grid = MultiGrid(width, height, torus=False)
        self.steps = 0

        # creamos la matriz de ceros
        self.dirt = np.zeros((width, height))

        # Activamos de forma aleatoria en los agentes
        self.schedule = RandomActivation(self)

        # Calculamos el número de celdas sucias
        num_dirty_cells = int(dirty_cells * width * height)

        # Crea una lista de celdas sucias
        matrix = [(x, y) for x in range(width) for y in range(height)]
        random.shuffle(matrix)
        dirty_cells = matrix[:num_dirty_cells]
        for cell in dirty_cells:
            self.dirt[cell] = 1  # Marca las celdas sucias en la matriz

        # Inicializamos nuestros agentes
        for i in range(num_agents):
            a = VacumAgent(i, self)  # Crea un nuevo agente
            # Coloca el agente en la posición (1,1)
            self.grid.place_agent(a, (1, 1))
            self.schedule.add(a)  # Añademos el agente a la matriz

    def count_dirt(self):
        # Cuenta el número de celdas sucias
        return sum(cell == 1 for cell in self.dirt.flatten())

    def is_dirt(self, pos):
        # Comprueba si una celda está sucia
        return self.dirt[pos[0], pos[1]] == 1

    def clean_dirt(self, pos):
        # Limpia una celda
        self.dirt[pos[0]][pos[1]] = 0

    def step(self):
        # Realiza un paso en la simulación
        self.schedule.step()
        self.steps += 1  # Incrementa el contador de pasos

In [4]:
# Esta función es para correr el modelo.
def run_model(grid_width, grid_height, agent_count, max_iterations, dirty_ratio):
    simulation_model = VacumModel(
        grid_width, grid_height, agent_count, max_iterations, dirty_ratio)

    # Esta parte es la que sustituye al ciclo while que me dijo el profesor que usara,
    # ya que lo que hago es tener un contador inicializado en 0 y que se vaya incrementando cada vez que se hace un step.
    # Entonces cuando termine de limpiar el porcentaje de celdas limpias que yo determine, se detiene el ciclo y me va 
    # a dar el número de iteraciones que se hicieron y de esta manera puedo saber el tiempo que se tardo en limpiar N celdas.
    iteration = 0
    while iteration < max_iterations and np.any(simulation_model.dirt == 1):
        simulation_model.step()
        iteration += 1
    return (
        iteration,
        (1 - np.mean(simulation_model.dirt)) * 100
    )

In [7]:
WIDTH = 100
HEIGHT = 100
NUM_AGENTS = 1
MAX_STEPS = 100000000000000
DIRTY_CELLS = 1

start_time = time.time()
result = run_model(WIDTH, HEIGHT, NUM_AGENTS, MAX_STEPS, DIRTY_CELLS)
total_time = time.time() - start_time

print("Tiempo total de la ejecución: ", total_time)
print("Número de pasos realizados: " + str(result[0]))
print("Porcentaje de celdas limpias: " + str(result[1]) + "%")

Tiempo total de la ejecución:  4.6254048347473145
Número de pasos realizados: 310565
Porcentaje de celdas limpias: 100.0%
